In [ ]:
import datasets
from src import data_loading

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.models.dn_embedding_model import DnEmbeddingModel
from src.models.optimizers.adamw_spec import AdamWSpec
from src.utils.data_split import ValidationSplit


model = DnEmbeddingModel(
    hidden_dims=[1024, 1024, 512, 512, 256, 256, 128, 128],
    optimizer_spec=AdamWSpec(
        learning_rate=1e-5,
        lr_decay_gamma=0.998,
    ),
    balance_model_samples=True,
    
    embedding_model_epochs=200,
    embedding_model_name="all-mpnet-base-v2",
    embedding_model_hidden_dims=[1024, 1024, 512, 512, 256, 256, 128, 128],
    embedding_model_optimizer_spec=AdamWSpec(
        learning_rate=5e-5,
        lr_decay_gamma=0.995,
    ),
    triplet_margin=0.2,
    regularization_weight=0,
    min_model_comparisons=1000,
    identity_positive_ratio=1,
    seed=42,
    print_every=5,
)

model.train(data, validation_split=ValidationSplit(val_fraction=0.2, seed=42), epochs=500, batch_size=2048)
model.save("longer-run-embedding-model-with-all-mpnet-base-v2")

print()
model.last_timer.inspect(1)
print()
model.embedding_model.last_timer.inspect(1)

In [ ]:
import matplotlib.pyplot as plt

from src.plotting_utils import plot_loss, plot_accuracy

history = model.get_history()
_, axes = plt.subplots(2, 2, figsize=(15, 10))

plot_loss(axes[0, 0], history.total_loss, "Training loss")
plot_loss(axes[0, 1], history.val_loss, "Validation loss")
plot_accuracy(axes[1, 0], history.train_accuracy, "Training accuracy")
plot_accuracy(axes[1, 1], history.val_accuracy, "Validation accuracy")

plt.tight_layout()
plt.show()

In [ ]:
embedding_history = model.embedding_model._epoch_logs

_, axes = plt.subplots(3, 2, figsize=(15, 15))

plot_loss(axes[0, 0], [e.train_loss for e in embedding_history], "Training loss")
plot_loss(axes[0, 1], [e.val_loss for e in embedding_history], "Validation loss")
plot_accuracy(axes[1, 0], [e.train_triplet_accuracy for e in embedding_history], "Training triplet accuracy")
plot_accuracy(axes[1, 1], [e.val_triplet_accuracy for e in embedding_history], "Validation triplet accuracy")

axes[2, 0].plot([e.train_loss for e in embedding_history], label="Training loss")
axes[2, 0].plot([e.train_triplet_loss for e in embedding_history], label="Training triplet loss")
axes[2, 0].plot([e.train_reg_loss for e in embedding_history], label="Training regularization loss")
axes[2, 0].legend()